# fds.datax:  getSDFdata Method Overview


The GetSDFData method is designed to streamline the retrieval of data from FactSet's Standard DataFeeds. Each attribute within the method is designed to retrieve a specific set of data items:

Main Features
--------------------

- Utilize FactSet's Ownership Database to access historical ETF holdings as a baseline universe. 

- Retrieve all levels of FactSet's Entity centric symbology model to all instance access to any content set within the Standard DataFeed ecosystem. 

- Access baseline categorical information such as country of domicile, sector classifications, and other company specific meta data. 

- Fetch prices (OHLC), 1 day total returns, market cap, exchange rates, and corporate action adjustment ratios for a universe. 


In [1]:
import fds.datax as dx
import os

# Inputs

The following inputs will be used for the remainder of the notebook. 

In [2]:
# ETF ticker in a ticker-region format.
etf_ticker = 'SPY-US'

# Earliest available report date in YYYY-MM-DD format
start_date = '2019-01-01'

# Most recent available report date to be returned in YYYY-MM-DD format
end_date = '2019-06-30'

#DSN name for a connection to a MSSQL Server DB containing FDS Standard DataFeeds content.
mssql_dsn = 'SDF'

#  currency for pricing data to be return, if local currency is desired set to "LOCAL"
currency = 'LOCAL'

____________________
etf_universe
-----------
Returns the constituents of an ETF from FDS Ownership in a pandas DataFrame.
ETF Ownership is available on a annual, semi-annual, quarterly, or monthly frequency
depending on the frequency of filings. Data will be resampled daily by forward filling the data set.


Parameters
-----------

   - etf_ticker: ETF ticker in a ticker-region format. For example,
                SPY-US.

   - start_date: earliest available report date in YYYY-MM-DD format

   - end_date: most recent available report date to be returned in YYYY-MM-DD format
   
   - mssql_dsn: DSN name for a connection to a MSSQL Server DB containing FDS Standard DataFeeds content.
   

Returns
-----------

A Pandas DataFrame containing: 

    ref_id | fsym_primary_listing_id | fsym_primary_equity_id | date | adj_holding 
   

In [3]:
etf_universe = dx.getSDFdata.etf_universe(
    etf_ticker=etf_ticker, start_date=start_date, end_date=end_date, mssql_dsn=mssql_dsn
)
etf_universe.head()

,ref_id,fsym_primary_listing_id,fsym_primary_equity_id,date
0,BZT3MC-S,RGTY61-R,BZT3MC-S,2019-01-31
1,BZT3MC-S,RGTY61-R,BZT3MC-S,2019-02-01
2,BZT3MC-S,RGTY61-R,BZT3MC-S,2019-02-02
3,BZT3MC-S,RGTY61-R,BZT3MC-S,2019-02-03
4,BZT3MC-S,RGTY61-R,BZT3MC-S,2019-02-04


____________________

fds_symbology
-----------

Returns FDS symbology at the -R, -S, and -E level.  For the -E will be returned in a PIT
manner based on the date in the input data frame.

Parameters
-----------

   - univ_df: the input dataframe containing at least a FDS or market ID
            (SEDOL, CUSIP, ISIN all with a check digit included)

   - mssql_dsn: DSN name for a connection to a MSSQL Server DB containing
                 FDS Standard DataFeeds content.

   - id_type: flagged designating if the input ID is a FDS ID or market
            1: Market ID (SEDOL, CUSIP, ISIN all must include a check digit)
            2: FDS Id

   - ref_id: This will identify the column name in the univ_df of the primary
            ID.

   - ref_date: This will identify the column name in the univ_df of the date field.

Returns
-----------

A Pandas DataFrame containing: 

    ref_id | fsym_primary_listing_id | fsym_primary_equity_id | date  | proper_name | factset_entity_id

In [4]:
fds_symbology = dx.getSDFdata.fds_symbology(
    univ_df=etf_universe,
    mssql_dsn=mssql_dsn,
    id_type=0,
    ref_id="ref_id",
    ref_date="date",
)

fds_symbology.head()

,ref_id,fsym_primary_listing_id,fsym_primary_equity_id,date,proper_name,factset_entity_id
1,BZT3MC-S,RGTY61-R,BZT3MC-S,2019-01-31,"Fidelity National Information Services, Inc.",06CVZ2-E
3,BZT3MC-S,RGTY61-R,BZT3MC-S,2019-02-01,"Fidelity National Information Services, Inc.",06CVZ2-E
5,BZT3MC-S,RGTY61-R,BZT3MC-S,2019-02-02,"Fidelity National Information Services, Inc.",06CVZ2-E
7,BZT3MC-S,RGTY61-R,BZT3MC-S,2019-02-03,"Fidelity National Information Services, Inc.",06CVZ2-E
9,BZT3MC-S,RGTY61-R,BZT3MC-S,2019-02-04,"Fidelity National Information Services, Inc.",06CVZ2-E


____________________

fds_sec_ref
-----------------

Returns country and RBICS sector levels 1-4 for a universe.

Parameters
-----------

   - entity_id_list:  A python list containing FactSet Entity IDs for the universe.**

   - mssql_dsn: DSN name for a connection to a MSSQL Server DB containing
                 FDS Standard DataFeeds content.

**To retrieve FactSet Entity IDs please use the fds_symbology function.

Returns
-----------

A Pandas DataFrame containing:

    factset_entity_id | entity_proper_name | country | region | rbics_l1_id |rbics_l1 |....| rbics_l4_id | rbics_l4


In [5]:
entity_ids = fds_symbology.factset_entity_id.unique().tolist()

fds_sec_ref = dx.getSDFdata.fds_sec_ref(entity_id_list=entity_ids, mssql_dsn=mssql_dsn)

fds_sec_ref.head()

,factset_entity_id,entity_proper_name,country,region,rbics_l1_id,rbics_l1,rbics_l2_id,rbics_l2,rbics_l3_id,rbics_l3,rbics_l4_id,rbics_l4
0,003196-E,Garmin Ltd.,Switzerland,Europe,55,Technology,5515,Hardware,551515,Communications Equipment,55151530,Satellite Equipment
1,0G0L3Q-E,Chubb Ltd.,Switzerland,Europe,30,Finance,3015,Insurance,301510,Insurance,30151020,Property and Casualty Insurance
2,05VFN0-E,TE Connectivity Ltd.,Switzerland,Europe,55,Technology,5510,Electronic Components and Manufacturing,551010,Electronic Components,55101030,Diversified Electronic Components
3,00D7D5-E,Aptiv Plc,Ireland,Europe,20,Consumer Cyclicals,2020,Consumer Vehicles and Parts,202010,Consumer Vehicles and Parts,20201010,Consumer Vehicle Parts Manufacturing
4,091B1B-E,Allergan Plc,Ireland,Europe,35,Healthcare,3510,Biopharmaceuticals,351015,Other Biopharmaceuticals,35101540,Diversified Biopharmaceuticals


____________________

fds_prices
-------------

Returns daily pricing information for the specified universe. 

Parameters
-----------

   - regional_id_list:  A python list containing FactSet Entity IDs for the universe.**

   - start_date:  The earliest date for which pricing will be retrieved.  Format requires
             YYYY-MM-DD.
             
   - end_date:  The earliest date for which pricing will be retrieved.  Format requires
             YYYY-MM-DD.
             
   - currency: currency for pricing data to be return, if local currency is desired set
           to "LOCAL"

   - adjtype:  Specifies the type of pricing to be returned:

          0:  Unadjusted values.
          1:  Split adjusted values.
          2:  Split and spin-off adjusted values.
          3:  All types.

   - mssql_dsn: DSN name for a connection to a MSSQL Server DB containing
                 FDS Standard DataFeeds content.

**To retrieve FactSet Entity IDs please use the fds_symbology method.

Returns
-----------

A Pandas DataFrame containing:

    fsym_id | price_date | currency | one_day_total_return | volume | shares_outstanding | price_close | price_high | price_low | price_open | market_value

In [6]:
regional_ids = fds_symbology.fsym_primary_listing_id.unique().tolist()

fds_prices = dx.getSDFdata.fds_prices(
    regional_id_list=regional_ids,
    start_date=start_date,
    end_date=end_date,
    currency=currency,
    adjtype=2,
    mssql_dsn=mssql_dsn,
)

fds_prices.head()

,fsym_id,price_date,currency,one_day_total_return,volume,shares_outstanding,price_close,price_high,price_low,price_open,market_value
0,B71DJZ-R,2019-01-01,USD,0.000000,0.000,156743.0,24.74,0.00,0.00,0.00,3877821.82
1,B71DJZ-R,2019-01-02,USD,4.405820,4595.546,156743.0,25.83,25.85,24.11,24.37,4048671.69
2,B71DJZ-R,2019-01-03,USD,-0.580716,4947.926,156743.0,25.68,25.95,24.97,25.47,4025160.24
3,B71DJZ-R,2019-01-04,USD,1.791270,3856.144,156743.0,26.14,26.42,25.91,25.97,4097262.02
4,B71DJZ-R,2019-01-07,USD,1.491980,4023.104,156743.0,26.53,26.83,25.80,26.09,4158391.79


____________________

fds_fx_rates
-----------------

Returns daily exchange rates from a list of currency to a common target currency. 

Parameters
-----------

    - currency_list:  A python list containing currency ISO-3

    - target_currency: Currency to be converted to (ISO-3)
    
    - start_date: earliest fx rate to be returned in YYYY-MM-DD format

    - end_date: most recent fx rate to be returned in YYYY-MM-DD format

    - mssql_dsn: DSN name for a connection to a MSSQL Server DB containing
                 FDS Standard DataFeeds content.

Returns
-----------

A Pandas DataFrame containing:

    price_date | currency | currency_to | exch_rate_usd | exch_rate_per_usd_to | fx_rate
    

In [7]:
currs = fds_prices.currency.unique().tolist()

target_currency = "EUR"


fds_fx_rates = dx.getSDFdata.fds_fx_rates(
    currency_list=currs,
    target_currency=target_currency,
    start_date=start_date,
    end_date=end_date,
    mssql_dsn=mssql_dsn,
)

fds_fx_rates.sort_values(["currency", "price_date"], ascending=False).head()

,price_date,currency,currency_to,exch_rate_usd,exch_rate_per_usd_to,fx_rate
357,2019-06-30,USD,EUR,1.0,0.878117,0.878117
329,2019-06-29,USD,EUR,1.0,0.878117,0.878117
297,2019-06-28,USD,EUR,1.0,0.878117,0.878117
239,2019-06-27,USD,EUR,1.0,0.879624,0.879624
312,2019-06-26,USD,EUR,1.0,0.879005,0.879005


____________________

fds_corp_actions
-----------------

Returns cumulative adjustment factors for splits and spin-offs for a universe. 

Parameters
-----------

    - regional_id_list:  A python list containing FactSet Entity IDs for the universe.

    - mssql_dsn: DSN name for a connection to a MSSQL Server DB containing
                 FDS Standard DataFeeds content.
                 
**To retrieve FactSet Regional IDs please use the fds_symbology method.

Returns
-----------

A Pandas DataFrame containing:

    fsym_id | price_date | price_end_date | cum_split_factor | cum_spin_factor

In [8]:
fds_corp_actions = dx.getSDFdata.fds_corp_actions(
    regional_id_list=regional_ids, mssql_dsn=mssql_dsn
)

fds_corp_actions.head()

,fsym_id,price_date,price_end_date,cum_spin_factor,cum_split_factor
0,M0C1P8-R,1975-01-08,1900-01-01,NaN,0.023992
1,BV8JHF-R,1975-01-21,1900-01-01,0.924504,0.058895
2,QSDQCP-R,1975-03-03,1900-01-01,0.749581,0.083333
3,G7VZQH-R,1975-03-14,1900-01-01,NaN,0.030799
4,F17SJ1-R,1975-03-17,1900-01-01,NaN,0.010049
